In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) #See all dataframe columns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
columns = ['game_id','game_date','season','season_type','week',
           'home_team','away_team','spread_line','result',
           'game_half','qtr','drive','play_id','play_type','desc',
           'posteam','defteam',
           'down','ydstogo','yardline_100','score_differential',
           'half_seconds_remaining','game_seconds_remaining',
           'posteam_timeouts_remaining','defteam_timeouts_remaining',
           'ep','wp']

In [4]:
#Enter desired years of data
start = 1999
end = 2020

data = pd.DataFrame()

for i in range(start,end+1):  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         usecols=columns, compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

In [5]:
data.shape
data.info()
data.head()

(1047858, 27)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047858 entries, 0 to 1047857
Data columns (total 27 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   away_team                   1047858 non-null  object 
 1   defteam                     976890 non-null   object 
 2   defteam_timeouts_remaining  976890 non-null   float64
 3   desc                        1047858 non-null  object 
 4   down                        872305 non-null   float64
 5   drive                       1034305 non-null  float64
 6   ep                          1030870 non-null  float64
 7   game_date                   1047858 non-null  object 
 8   game_half                   1047858 non-null  object 
 9   game_id                     1047858 non-null  object 
 10  game_seconds_remaining      1046663 non-null  float64
 11  half_seconds_remaining      1046663 non-null  float64
 12  home_team                   1047858 non-null  object 
 1

,away_team,defteam,defteam_timeouts_remaining,desc,down,drive,ep,game_date,game_half,game_id,game_seconds_remaining,half_seconds_remaining,home_team,play_id,play_type,posteam,posteam_timeouts_remaining,qtr,result,score_differential,season,season_type,spread_line,week,wp,yardline_100,ydstogo
0,ARI,ARI,3.0,C.Jacke kicks 68 yards from ARI 30 to PHI 2. A...,NaN,1.0,0.637888,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,35,kickoff,PHI,3.0,1,-1,NaN,1999,REG,-3.0,1,0.000294,30.0,0
1,ARI,ARI,3.0,(15:00) D.Pederson pass incomplete to D.Staley...,1.0,1.0,0.764706,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,60,pass,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.546969,77.0,10
2,ARI,ARI,3.0,(15:00) D.Staley up the middle to PHI 24 for 1...,2.0,1.0,0.203138,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,82,run,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.528741,77.0,10
3,ARI,ARI,3.0,"(15:00) PENALTY on PHI-K.Turner, False Start, ...",3.0,1.0,-0.438578,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,103,no_play,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.498319,76.0,9
4,ARI,ARI,3.0,(15:00) D.Pederson pass to B.Finneran to PHI 2...,3.0,1.0,-1.161881,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,126,pass,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.490872,81.0,14


In [6]:
adj = data.copy(deep=True)

In [7]:
# Remove OT games.
ot_games = adj['game_id'][adj['qtr']==5].unique()
adj = adj[~adj['game_id'].isin(ot_games)].reset_index(drop=True)

In [8]:
# Remove tied games. Not necessary if you've already removed OT games.
adj = adj[adj['result']!=0].reset_index(drop=True)

In [9]:
# Remove non-plays.
adj = adj[~adj['posteam'].isnull()].reset_index(drop=True)
adj = adj[~adj['play_type'].isnull()].reset_index(drop=True)

In [10]:
# Create the "label" field.
adj['Winner'] = np.where(adj['result'] > 0, adj['home_team'], adj['away_team'])
adj['label'] = np.where(adj['Winner'] == adj['posteam'], 1, 0)

In [11]:
# Create the "receive_2h_ko" field.
receive_2h_ko = adj.groupby('game_id')['defteam'].first().reset_index()
receive_2h_ko.rename(columns={"defteam":"receive_2h_ko"},inplace=True)

adj = adj.merge(receive_2h_ko, left_on='game_id', right_on='game_id')
adj['receive_2h_ko'] = np.where((adj['posteam'] == adj['receive_2h_ko']) & (adj['game_half'] == 'Half1'), 1, 0)

In [12]:
# Create the "home" field.
adj['home'] = np.where(adj['posteam'] == adj['home_team'], 1, 0)

In [13]:
# Create the "posteam_spread" field.
adj['posteam_spread'] = np.where(adj['home'] == 1, adj['spread_line'], -1*adj['spread_line'])

In [14]:
# Create the "elapsed_share" field.
adj['elapsed_share'] = (3600 - adj['game_seconds_remaining']) / 3600

In [15]:
# Create the "spread_time" field.
adj['spread_time'] = adj['posteam_spread'] * (np.exp(-4 * adj['elapsed_share']))

In [16]:
# Create the "Diff_Time_Ratio" field.
adj['Diff_Time_Ratio'] = adj['score_differential'] / (np.exp(-4 * adj['elapsed_share']))

In [17]:
adj.shape
adj.info()
adj.head()

(897589, 35)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897589 entries, 0 to 897588
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   away_team                   897589 non-null  object 
 1   defteam                     897589 non-null  object 
 2   defteam_timeouts_remaining  897589 non-null  float64
 3   desc                        897589 non-null  object 
 4   down                        812335 non-null  float64
 5   drive                       895536 non-null  float64
 6   ep                          897589 non-null  float64
 7   game_date                   897589 non-null  object 
 8   game_half                   897589 non-null  object 
 9   game_id                     897589 non-null  object 
 10  game_seconds_remaining      897584 non-null  float64
 11  half_seconds_remaining      897584 non-null  float64
 12  home_team                   897589 non-null  object 
 13  play_id       

,away_team,defteam,defteam_timeouts_remaining,desc,down,drive,ep,game_date,game_half,game_id,game_seconds_remaining,half_seconds_remaining,home_team,play_id,play_type,posteam,posteam_timeouts_remaining,qtr,result,score_differential,season,season_type,spread_line,week,wp,yardline_100,ydstogo,Winner,label,receive_2h_ko,home,posteam_spread,elapsed_share,spread_time,Diff_Time_Ratio
0,ARI,ARI,3.0,C.Jacke kicks 68 yards from ARI 30 to PHI 2. A...,NaN,1.0,0.637888,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,35,kickoff,PHI,3.0,1,-1,NaN,1999,REG,-3.0,1,0.000294,30.0,0,ARI,0,0,1,-3.0,0.0,-3.0,NaN
1,ARI,ARI,3.0,(15:00) D.Pederson pass incomplete to D.Staley...,1.0,1.0,0.764706,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,60,pass,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.546969,77.0,10,ARI,0,0,1,-3.0,0.0,-3.0,0.0
2,ARI,ARI,3.0,(15:00) D.Staley up the middle to PHI 24 for 1...,2.0,1.0,0.203138,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,82,run,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.528741,77.0,10,ARI,0,0,1,-3.0,0.0,-3.0,0.0
3,ARI,ARI,3.0,"(15:00) PENALTY on PHI-K.Turner, False Start, ...",3.0,1.0,-0.438578,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,103,no_play,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.498319,76.0,9,ARI,0,0,1,-3.0,0.0,-3.0,0.0
4,ARI,ARI,3.0,(15:00) D.Pederson pass to B.Finneran to PHI 2...,3.0,1.0,-1.161881,1999-09-12,Half1,1999_01_ARI_PHI,3600.0,1800.0,PHI,126,pass,PHI,3.0,1,-1,0.0,1999,REG,-3.0,1,0.490872,81.0,14,ARI,0,0,1,-3.0,0.0,-3.0,0.0


In [18]:
adj.to_csv('pbp_data.csv', index=False)